## Line Following

<img align="right" src="img/line_following.png" />
Up to now, we have worked up a line detection algorithm. Now that this scheme is up an running, we can move on to the task of driving the robot such that the line stays near the center of the camera image.

We propose to use [a *proportional* controller](https://en.wikipedia.org/wiki/Proportional_control), which means that a linear scaling of an error drives the control output. In this case, the error signal is the distance between the center of the image and the center of the line that we are trying to follow. The control output is the steering (angular velocity) of the robot.


In [103]:
import packages.initialization
import pioneer3dx as p3dx
p3dx.init()

In [104]:
import cv2
import numpy

### Image processing
Fill in the necessary code in the following function, which computes the centroid of the line of the image passed as an argument, as explained in the previous notebook.

In [105]:
def check_hsv_param(hsv_value, upper_limit, hsv_range):
    lower_value = hsv_value - hsv_range
    if lower_value < 0:
        lower_value = 0
    elif lower_value > upper_limit:
        lower_value = upper_limit
        
    upper_value = hsv_value + hsv_range
    if upper_value < 0:
        upper_value = 0
    elif upper_value > upper_limit:
        upper_value = upper_limit
        
    return lower_value, upper_value
    

def line_centroid(image):
    lower_hue, upper_hue = check_hsv_param(hue, 180, hue_range)
    lower_saturation, upper_saturation = check_hsv_param(saturation, 255, saturation_range)
    lower_value, upper_value = check_hsv_param(value, 255, value_range)
    
    print("%d %d %d %d %d %d" % (lower_hue, upper_hue, lower_saturation, upper_saturation, lower_value, upper_value))
    
    lower_color = numpy.array([lower_hue, lower_saturation, lower_value])
    upper_color = numpy.array([upper_hue, upper_saturation, upper_value])
    
    print(lower_color)
    print(upper_color)
    
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    mask[0:80, 0:150] = 0
    
    M = cv2.moments(mask)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    
    return cx, cy

We need the code for the motion of the robot with the given linear and angular velocities, as in previous modules.

In [106]:
def move(v_robot, w_robot):    
    w_l = (2*v_robot - axis_length*w_robot) / (2*wheel_radius)
    w_r = (2*v_robot + axis_length*w_robot) / (2*wheel_radius)    
    p3dx.move(w_l, w_r)

### Main loop
This is the main control loop. The error should be computed as:

$$ err = C_x - \frac{width}{2} $$

where $C_x$ is the $x$-coordinate of the centroid, and $width$ is the width of the image.

The linear velocity is constant, e.g. $2 m/s$ and the angular velocity $\omega$ is computed as:

$$ \omega = -K_p err $$

where $K_p$ is the gain of the proportional controller, which can be set to $0.01$.

In [107]:
wheel_diameter = 0.1953         # In meters
wheel_radius = wheel_diameter/2 # In meters
axis_length = 0.33              # In meters

# cyan color in hsv format
hue = 90       
hue_range = 10

saturation = 255  
saturation_range = 155

value = 255
value_range = 155

p3dx.tilt(-0.47) # tilt down the Kinect
try:
    _, width, _ = p3dx.image.shape
    print("width: %d" % width)
    # It's important to have a high angular velocity, otherwise the line is lost in the image.
    # The higher the linear velocity the higher the kp must be in order to increase the angular velocity
    # and avoid losing the line in the image.
    v_robot = 2 # v_robot = 1 along with kp = 0.05
    kp = 0.1
    while True:
        cx, cy = line_centroid(p3dx.image)
        print("cx: %f" % cx)
        err = cx - width/2
        print("error: %f" % err)        
        w_robot = -kp * err
        print("w_robot: %f" % w_robot)
        move(v_robot, w_robot)
        #p3dx.sleep(1)
        #p3dx.stop()
        #v = input("Presiona una tecla")
except KeyboardInterrupt:
    move(0,0)
    #plt.imshow(image)
    #axes = plt.gca()
    #axes.add_artist(plt.Circle((cx,cy),10,color='r'));

width: 150
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 20.000000
error: -55.000000
w_robot: 5.500000
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 18.000000
error: -57.000000
w_robot: 5.700000
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 31.000000
error: -44.000000
w_robot: 4.400000
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 41.000000
error: -34.000000
w_robot: 3.400000
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 48.000000
error: -27.000000
w_robot: 2.700000
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 55.000000
error: -20.000000
w_robot: 2.000000
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 60.000000
error: -15.000000
w_robot: 1.500000
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 65.000000
error: -10.000000
w_robot: 1.000000
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 67.000000
error: -8.000000
w_robot: 0.800000
80 100 100 255 100 255
[ 80 100 100]
[100 255 255]
cx: 69.000000
error: -6.000000

ZeroDivisionError: float division by zero

Next: [Line Following with Obstacle Avoidance](Line%20Following%20Obstacle.ipynb)

---
#### Try-a-Bot: an open source guide for robot programming
Developed by:
[![Robotic Intelligence Lab @ UJI](img/logo/robinlab.png "Robotic Intelligence Lab @ UJI")](http://robinlab.uji.es)

Sponsored by:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="http://www.ieee-ras.org"><img src="img/logo/ras.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.cyberbotics.com"><img src="img/logo/cyberbotics.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.theconstructsim.com"><img src="img/logo/theconstruct.png"></a>
</td>
</tr>
</table>

Follow us:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="https://www.facebook.com/RobotProgrammingNetwork"><img src="img/logo/facebook.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="https://www.youtube.com/user/robotprogrammingnet"><img src="img/logo/youtube.png"></a>
</td>
</tr>
</table>